In [1]:
import gc
from tqdm.notebook import tqdm

import pandas as pd
import numpy as np
from sklearn.preprocessing import LabelEncoder, FunctionTransformer
from sklearn.pipeline import make_pipeline
from sklearn.compose import make_column_transformer
from sklearn.model_selection import train_test_split
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.impute import SimpleImputer 

import keras
from keras.models import Model
from keras.utils.vis_utils import model_to_dot
from keras.layers import Input, Dense, Dropout, BatchNormalization
import tensorflow as tf


Using TensorFlow backend.


## Data loading

In [2]:
folder_path = '/Users/pawelurbanowicz/master-thesis/data/'

train_identity = pd.read_csv(f'{folder_path}train_identity.csv')
train_transaction = pd.read_csv(f'{folder_path}train_transaction.csv')

# train = pd.merge(train_transaction, train_identity, on='TransactionID', how='left')
train = train_transaction

## Data exploration 

In [3]:
print('train: ', train.shape)

train:  (590540, 394)


In [4]:
train_transaction.head(3)

,TransactionID,isFraud,TransactionDT,TransactionAmt,ProductCD,card1,card2,card3,card4,card5,...,V330,V331,V332,V333,V334,V335,V336,V337,V338,V339
0,2987000,0,86400,68.5,W,13926,NaN,150.0,discover,142.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2987001,0,86401,29.0,W,2755,404.0,150.0,mastercard,102.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2987002,0,86469,59.0,W,4663,490.0,150.0,visa,166.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [5]:
train_identity.head(3)

,TransactionID,id_01,id_02,id_03,id_04,id_05,id_06,id_07,id_08,id_09,...,id_31,id_32,id_33,id_34,id_35,id_36,id_37,id_38,DeviceType,DeviceInfo
0,2987004,0.0,70787.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,samsung browser 6.2,32.0,2220x1080,match_status:2,T,F,T,T,mobile,SAMSUNG SM-G892A Build/NRD90M
1,2987008,-5.0,98945.0,NaN,NaN,0.0,-5.0,NaN,NaN,NaN,...,mobile safari 11.0,32.0,1334x750,match_status:1,T,F,F,T,mobile,iOS Device
2,2987010,-5.0,191631.0,0.0,0.0,0.0,0.0,NaN,NaN,0.0,...,chrome 62.0,NaN,NaN,NaN,F,F,T,T,desktop,Windows


In [6]:
del train_identity, train_transaction

## Feature enginnering 

In [7]:
# train['TransactionAmt_to_mean_card1'] = train['TransactionAmt'] / train.groupby(['card1'])['TransactionAmt'].transform('mean')
# train['TransactionAmt_to_mean_card4'] = train['TransactionAmt'] / train.groupby(['card4'])['TransactionAmt'].transform('mean')
# train['TransactionAmt_to_std_card1'] = train['TransactionAmt'] / train.groupby(['card1'])['TransactionAmt'].transform('std')
# train['TransactionAmt_to_std_card4'] = train['TransactionAmt'] / train.groupby(['card4'])['TransactionAmt'].transform('std')

# test['TransactionAmt_to_mean_card1'] = test['TransactionAmt'] / test.groupby(['card1'])['TransactionAmt'].transform('mean')
# test['TransactionAmt_to_mean_card4'] = test['TransactionAmt'] / test.groupby(['card4'])['TransactionAmt'].transform('mean')
# test['TransactionAmt_to_std_card1'] = test['TransactionAmt'] / test.groupby(['card1'])['TransactionAmt'].transform('std')
# test['TransactionAmt_to_std_card4'] = test['TransactionAmt'] / test.groupby(['card4'])['TransactionAmt'].transform('std')

# train['id_02_to_mean_card1'] = train['id_02'] / train.groupby(['card1'])['id_02'].transform('mean')
# train['id_02_to_mean_card4'] = train['id_02'] / train.groupby(['card4'])['id_02'].transform('mean')
# train['id_02_to_std_card1'] = train['id_02'] / train.groupby(['card1'])['id_02'].transform('std')
# train['id_02_to_std_card4'] = train['id_02'] / train.groupby(['card4'])['id_02'].transform('std')

# test['id_02_to_mean_card1'] = test['id_02'] / test.groupby(['card1'])['id_02'].transform('mean')
# test['id_02_to_mean_card4'] = test['id_02'] / test.groupby(['card4'])['id_02'].transform('mean')
# test['id_02_to_std_card1'] = test['id_02'] / test.groupby(['card1'])['id_02'].transform('std')
# test['id_02_to_std_card4'] = test['id_02'] / test.groupby(['card4'])['id_02'].transform('std')

# train['D15_to_mean_card1'] = train['D15'] / train.groupby(['card1'])['D15'].transform('mean')
# train['D15_to_mean_card4'] = train['D15'] / train.groupby(['card4'])['D15'].transform('mean')
# train['D15_to_std_card1'] = train['D15'] / train.groupby(['card1'])['D15'].transform('std')
# train['D15_to_std_card4'] = train['D15'] / train.groupby(['card4'])['D15'].transform('std')

# test['D15_to_mean_card1'] = test['D15'] / test.groupby(['card1'])['D15'].transform('mean')
# test['D15_to_mean_card4'] = test['D15'] / test.groupby(['card4'])['D15'].transform('mean')
# test['D15_to_std_card1'] = test['D15'] / test.groupby(['card1'])['D15'].transform('std')
# test['D15_to_std_card4'] = test['D15'] / test.groupby(['card4'])['D15'].transform('std')

# train['D15_to_mean_addr1'] = train['D15'] / train.groupby(['addr1'])['D15'].transform('mean')
# train['D15_to_mean_addr2'] = train['D15'] / train.groupby(['addr2'])['D15'].transform('mean')
# train['D15_to_std_addr1'] = train['D15'] / train.groupby(['addr1'])['D15'].transform('std')
# train['D15_to_std_addr2'] = train['D15'] / train.groupby(['addr2'])['D15'].transform('std')

# test['D15_to_mean_addr1'] = test['D15'] / test.groupby(['addr1'])['D15'].transform('mean')
# test['D15_to_mean_addr2'] = test['D15'] / test.groupby(['addr2'])['D15'].transform('mean')
# test['D15_to_std_addr1'] = test['D15'] / test.groupby(['addr1'])['D15'].transform('std')
# test['D15_to_std_addr2'] = test['D15'] / test.groupby(['addr2'])['D15'].transform('std')

In [8]:
# train[['P_emaildomain_1', 'P_emaildomain_2', 'P_emaildomain_3']] = train['P_emaildomain'].str.split('.', expand=True)
# train[['R_emaildomain_1', 'R_emaildomain_2', 'R_emaildomain_3']] = train['R_emaildomain'].str.split('.', expand=True)
# test[['P_emaildomain_1', 'P_emaildomain_2', 'P_emaildomain_3']] = test['P_emaildomain'].str.split('.', expand=True)
# test[['R_emaildomain_1', 'R_emaildomain_2', 'R_emaildomain_3']] = test['R_emaildomain'].str.split('.', expand=True)

## Data preparation 

In [9]:
X = train.sort_values('TransactionDT').drop(['isFraud'], axis=1)
y = train.sort_values('TransactionDT')['isFraud']

In [10]:
categorical_features = ['ProductCD','P_emaildomain','R_emaildomain',
                'card1','card2','card3','card4','card5','card6',
                'addr1','addr2',
                'M1','M2','M3','M4','M5','M6','M7','M8','M9']

one_value_cols = [col for col in X.columns if X[col].nunique() <= 1]
many_null_cols = [col for col in X.columns if X[col].isnull().sum() / X.shape[0] > 0.9]
big_top_value_cols = [col for col in X.columns if X[col].value_counts(dropna=False, normalize=True).values[0] > 0.9]
cols_to_drop = list(set(many_null_cols + big_top_value_cols + one_value_cols + ['TransactionDT', 'TransactionID']))

from utils.tranformers import drop_columns, factorize

columns = list(set(X.columns) - set(cols_to_drop + categorical_features))

transformer = make_column_transformer(
    (FunctionTransformer(drop_columns, validate=False), cols_to_drop),
    (SimpleImputer(missing_values=np.nan, strategy='constant',fill_value=0), columns),
    (FunctionTransformer(factorize, validate=False), categorical_features),
    remainder='passthrough',
    verbose=True
)

X = transformer.fit_transform(X)


[ColumnTransformer]  (1 of 3) Processing functiontransformer-1, total=   0.0s
[ColumnTransformer] . (2 of 3) Processing simpleimputer, total=   8.8s
[ColumnTransformer]  (3 of 3) Processing functiontransformer-2, total=   0.8s


In [11]:
del train

## Model building 

In [12]:
inputs = Input(shape=(X.shape[1],))
dense_1 = Dense(10, activation='relu')(inputs)
dense_2 = Dense(10, activation='relu')(dense_1)
outputs = Dense(1, activation='sigmoid')(dense_2)
    
model = Model(inputs=inputs, outputs=outputs)
model.compile(
    loss='binary_crossentropy', 
    optimizer='adam', 
    metrics=['acc']
)

Instructions for updating:
If using Keras pass *_constraint arguments to layers.
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


In [13]:
import sys
import numpy
numpy.set_printoptions(threshold=sys.maxsize)

print(X[:1,:])
history = model.fit(
    x=X, 
    y=y, 
    validation_split=0.2, 
    epochs=10, 
    batch_size=128)

[[  0.    0.    1.    1.    0.    0.    0.    0.    0.    1.    1.    0.
    0.    0.    0.    0.    0.    0.    0.    0.    0.    0.    0.    1.
   13.    0.    0.    0.    1.    0.    0.    0.    1.    0.    0.    0.
    0.    0.    0.    1.    0.    0.    0.    0.    0.    0.    0.   19.
    0.    0.    1.    0.    0.    0.    0.    0.    0.    0.    0.    0.
    0.    0.    2.    0.   68.5   1.    0.    0.    0.    0.    1.    0.
    1.    0.    0.    0.    0.    1.    0.    0.    0.    0.    0.    0.
    0.    0.    1.    0.    0.    0.    0.    1.    0.    0.    0.    0.
    0.    0.    0.    0.    0.    0.    0.    0.    0.    0.    0.    0.
    0.    0.    0.    1.    0.    0.    0.    0.    0.    0.    0.    0.
    0.    0.    1.    1.    0.    0.    0.    0.    1.   13.    0.    0.
    0.    0.    0.    0.    0.    0.    0.    0.    0.    0.    0.    0.
    0.    0.    0.    0.    0.    0.    0.    1.    0.    0.  117.    0.
    0.    0.    0.    0.    0.    0.   14.    1.   

## Saving model 

In [14]:
import joblib

joblib.dump(transformer, "assets/data_transformer.joblib")
model.save("assets/fraud_prediction_model.h5")



1.15.0
